In [392]:
import sqlalchemy as db
from sqlalchemy.orm import Session, sessionmaker
from sqlalchemy import select, join

In [393]:
db.__version__

'2.0.27'

In [394]:
engine = db.create_engine('sqlite:///test_db.db')

In [395]:
conn = engine.connect()

In [396]:
metadata = db.MetaData()

In [397]:
#dir(db)

In [398]:
deposits = db.Table(
    'Deposit', metadata,
    db.Column('id_crt', db.Integer, primary_key = True),
    db.Column('id_clnt', db.Integer),
    db.Column('id_currency', db.Integer),
    db.Column('cur_perc', db.Integer),
    db.Column('rest_od', db.Integer),
    db.Column('rest_od_eq', db.Integer),    
    db.Column('id_prod', db.Integer),
    db.Column('id_cbu', db.Integer),
    )

currency = db.Table(
    'Currency', metadata,
    db.Column('id_currency', db.Integer, primary_key = True),
    db.Column('number_curr', db.Integer),
    )

clients = db.Table(
    'Client', metadata,
    db.Column('id_clnt', db.Integer, primary_key = True),
    db.Column('clnt_internal_num', db.Integer),
    db.Column('name', db.Text),
    db.Column('adress', db.Text),
    db.Column('reg_number', db.Integer),
    )

loans = db.Table(
    'Loan', metadata,
    db.Column('id_crt', db.Integer, primary_key = True),
    db.Column('id_clnt', db.Integer),
    db.Column('id_currency', db.Integer),
    db.Column('type_loan', db.Text),
    db.Column('cur_percent', db.Integer),
    db.Column('rest_od', db.Integer),
    db.Column('rest_od_eq', db.Integer)    
    )

In [399]:
metadata.create_all(engine)

currency code 4 - 933
currency code 3 - 840 - 3.2179
currency code 2 - 978 - 3.492 
currency code 1 - 643 - 0,035417

In [400]:
insertion_1 = deposits.insert().values([
    {'id_clnt': 2, 'id_currency': 4, 'cur_perc': 2, 'rest_od': 1200, 'rest_od_eq': 1200, 'id_prod': 1, 'id_cbu': 3},
    {'id_clnt': 3, 'id_currency': 2, 'cur_perc': 1, 'rest_od': 100, 'rest_od_eq': 349.2, 'id_prod': 2, 'id_cbu': 2},
    {'id_clnt': 1, 'id_currency': 4, 'cur_perc': 7, 'rest_od': 240, 'rest_od_eq': 240, 'id_prod': 3, 'id_cbu': 1},
    {'id_clnt': 4, 'id_currency': 3, 'cur_perc': 15, 'rest_od': 1000, 'rest_od_eq': 3492, 'id_prod': 4, 'id_cbu': 3}
])

In [401]:
conn.execute(insertion_1)

In [402]:
insertion_2 = currency.insert().values([
    {'number_curr': 643},
    {'number_curr': 978},
    {'number_curr': 840},
    {'number_curr': 933}
])

In [403]:
conn.execute(insertion_2)

In [404]:
insertion_3 = clients.insert().values([
    {'clnt_internal_num': 15432, 'name': 'ООО Ромашка', 'adress': 'г. Минск, ул. Кислева 31', 'reg_number': 75632},
    {'clnt_internal_num': 2746, 'name': 'ОАО Пекин', 'adress': 'г. Минск, ул. Калинина 53', 'reg_number': 5474},
    {'clnt_internal_num': 743, 'name': 'ЗАО Плюшевый медведь', 'adress': 'г. Минск, ул. Хабаровская 91', 'reg_number': 59483},
    {'clnt_internal_num': 40684, 'name': 'ОАО Тюльпан', 'adress': 'г. Минск, ул. Щукина 78', 'reg_number': 12749},
])

In [405]:
conn.execute(insertion_3)

In [406]:
insertion_4 = loans.insert().values([
    {'id_clnt': 3, 'id_currency': 4, 'type_loan': 'Возобновляемая кредитная линия', 'cur_percent': 6, 'rest_od': 1000, 'rest_od_eq': 1000},
    {'id_clnt': 2, 'id_currency': 3, 'type_loan': 'Кредитная линия', 'cur_percent': 10, 'rest_od': 1000, 'rest_od_eq': 3217.9},
    {'id_clnt': 4, 'id_currency': 2, 'type_loan': 'Невозобновляемая кредитная линия', 'cur_percent': 5, 'rest_od': 1000, 'rest_od_eq': 3492},
    {'id_clnt': 4, 'id_currency': 4, 'type_loan': 'Кредитная линия', 'cur_percent': 15, 'rest_od': 12000, 'rest_od_eq': 12000},
])

In [407]:
conn.execute(insertion_4)

In [412]:
select_all_query = db.select(deposits)
select_result = conn.execute(select_all_query)
select_result

In [413]:
select_result.fetchall()

[(1, 2, 4, 2, 1200, 1200, 1, 3),
 (2, 3, 2, 1, 100, 349.2, 2, 2),
 (3, 1, 4, 7, 240, 240, 3, 1),
 (4, 4, 3, 15, 1000, 3492, 4, 3)]

In [417]:
left_join = db.select(deposits, currency).where(deposits.c.id_currency == currency.c.id_currency)
result = conn.execute(left_join)
result.fetchall()

[(1, 2, 4, 2, 1200, 1200, 1, 3, 4, 933),
 (2, 3, 2, 1, 100, 349.2, 2, 2, 2, 978),
 (3, 1, 4, 7, 240, 240, 3, 1, 4, 933),
 (4, 4, 3, 15, 1000, 3492, 4, 3, 3, 840)]